# 建立和训练模型 #

$$f('真好吃')=1$$
$$f('太难吃')=0$$

**简介**
在已有的多种类的合成语料库的基础上，我们想训练一个可以预测语言情感的模型。

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import jieba # 结巴分词
# gensim用来加载预训练word vector
from gensim.models import KeyedVectors
import warnings
warnings.filterwarnings("ignore")

**预训练词向量**  
我们使用了北京师范大学中文信息处理研究所与中国人民大学 DBIIR 实验室的研究者开源的"chinese-word-vectors" github链接为：  
https://github.com/Embedding/Chinese-Word-Vectors  
我们选用其中的Zhihu_QA Word + Character + Ngram模型。

In [2]:
# 使用gensim加载预训练中文分词embedding
cn_model = KeyedVectors.load_word2vec_format('./sgns.zhihu.bigram-char', 
                                          binary=False)

在这个词向量模型里，每一个词是一个索引，对应的是一个长度为300的向量，我们今天构建的LSTM模型并不能直接处理汉字文本，需要先进行分次并把词汇转换为词向量。

In [3]:
# 由此可见每一个词都对应一个长度为300的向量
embedding_dim = cn_model['南方科技大学'].shape[0]
print('词向量的长度为{}'.format(embedding_dim))
cn_model['南方科技大学']

词向量的长度为300


array([ 6.10010e-02,  3.73003e-01, -2.71264e-01, -4.35535e-01,
       -6.01495e-01, -5.51020e-02,  2.70148e-01,  2.28470e-02,
        4.34196e-01,  1.59000e-02,  3.88832e-01,  3.25449e-01,
        3.55190e-01, -1.42635e-01,  2.43506e-01,  8.29120e-02,
        1.18370e-02,  5.00260e-01, -5.75470e-02,  1.35006e-01,
        6.13185e-01,  2.80526e-01,  1.72192e-01,  4.25960e-02,
       -1.32544e-01, -4.49176e-01, -6.74890e-02, -1.42176e-01,
       -7.92340e-02, -2.94032e-01, -2.34870e-01, -3.96320e-02,
       -1.49573e-01, -5.21620e-02,  5.70000e-04, -5.67600e-01,
        3.01528e-01,  2.15771e-01, -5.68168e-01,  3.69633e-01,
        2.38178e-01, -5.88849e-01,  9.80510e-02, -5.74732e-01,
        9.97770e-02,  5.06527e-01, -6.89000e-04,  1.12211e-01,
        4.56189e-01,  5.30550e-02, -3.90589e-01, -1.39010e-01,
       -1.20160e-02,  1.39989e-01,  2.07110e-01,  1.05418e-01,
       -5.12526e-01,  4.84178e-01, -4.09640e-02,  5.67858e-01,
        1.46660e-01,  6.70660e-02, -1.70841e-01, -6.495

有了词向量模型，我们可以轻松地寻找近义词、同类词。

In [5]:
# 计算相似度
cn_model.similarity('橘子', '橙子')

0.7151298

In [6]:
# 找出最相近的词，余弦相似度
cn_model.most_similar(positive=['大学'], topn=10)

[('本科', 0.6586681604385376),
 ('大学本科', 0.6412541270256042),
 ('大学毕业', 0.6330900192260742),
 ('高中', 0.6304237246513367),
 ('师范大学', 0.620072066783905),
 ('中学', 0.6154141426086426),
 ('读研', 0.5873822569847107),
 ('研究生', 0.5832617282867432),
 ('宾大', 0.5808302760124207),
 ('军校', 0.5738762021064758)]

In [7]:
# 找出不同的词
test_words = '老师 会计师 程序员 律师 医生 老人'
test_words_result = cn_model.doesnt_match(test_words.split())
print('在 '+test_words+' 中:\n不是同一类别的词为: %s' %test_words_result)

在 老师 会计师 程序员 律师 医生 老人 中:
不是同一类别的词为: 老人


In [19]:
Cat_Data = ['Neg_data_ed','Pos_data_ed']
path = './'
data_recorder = []

for i in range(2):
    print(path + Cat_Data[i] + '.txt')
    txt = open(path + Cat_Data[i] + '.txt', 'r', encoding='utf-8')
    while True:
        line = txt.read(1048)
        print(line,end="")
        if not line:
            break
    txt.close
    #data = txt.read().split('/n')
    #data_recorder.append([data,i])
print(data_recorder)

./Neg_data_ed.txt


UnsupportedOperation: not readable